In [1]:
# 将勒索地址的相关交易信息从文件中读取出来，经过处理之后，储存起来
from sklearn.feature_extraction import DictVectorizer
import csv
from sklearn import tree
from sklearn import preprocessing
from six import StringIO
import json
import pandas as pd
import os
from prettytable import PrettyTable

In [2]:
# 这个地址下是相关活动地址参与的交易
# path = r"/Users/linke/Desktop/myspace/git/ransomware/ransomware/ransomwareTxs"
# production_path = r"/Users/linke/Desktop/myspace/git/ransomware/production/transfer.csv"

path = r"R:/paper/git/ransomware/ransomware/ransomwareTxs/"
production_path = r"R:/paper/ransomware/production/"

In [3]:
# 用于存储最终结果
messages = []

In [4]:
def getAddrName(str):
    return str.split('_txs.json')[0]

In [5]:
def calculateVoutWeights(vouts):
    sum_vout = 0
    max_vout = 0
    for vout_item in vouts:
        sum_vout += vout_item['value']
        if vout_item['value'] > max_vout:
            max_vout = vout_item['value']
    return max_vout/sum_vout

In [6]:
def readFile(rootName, fileName):
    addrName = getAddrName(fileName)
    f = open(os.path.join(rootName, fileName))
    
    line = f.readline()
    while line:
        transaction_json = json.loads(line)
        result = handleTransJson(addrName, transaction_json)
        messages.append(result)
        line = f.readline()

    f.close()

In [7]:
def handleTransJson(addr, trans_json):
    ransom_message = []
    transfer_message = []
    
    for vin_item in trans_json['vin']:
        if(vin_item['address'] == addr):  
#             勒索地址作为输入
            transfer_message.append(trans_json['_id'])
            transfer_message.append(trans_json['timestamp'])
            transfer_message.append(trans_json['size'])
            
            transfer_message.append(trans_json['n_in'])
            transfer_message.append(trans_json['n_out'])
            
            transfer_message.append(calculateVoutWeights(trans_json['vout']))
            return [ransom_message, transfer_message]
            
    ransom_message.append(trans_json['_id'])
    ransom_message.append(trans_json['timestamp'])
    ransom_message.append(trans_json['size'])
    return [ransom_message, transfer_message]